In [1]:
# -------- IMPORTS --------
import numpy as np
from simulation.experiment import ForwardExperiment1D
from utility.distributions import spike, ricker, gaussian, raised_cosine, sinc, homogeneous, exponential, polynomial
from utility.plotting import plot_uv, plot_state, plot_medium, plot_heatmap

In [6]:
# Create experiment
experiment = ForwardExperiment1D(verbose=2)

# Define parameters
nx = 7
parameters_1 = {
    'dx': 1,
    'nx': nx, 
    'dt': 0.0001,
    'nt': 15,
    'order': 1,
    'bcs': {'left': 'DBC', 'right': 'DBC'},
    'mu': np.concatenate([homogeneous(3e10, nx+1)]),
    'rho': np.concatenate([homogeneous(3000, nx)]),
    'u': np.concatenate([ricker(1, nx, int(nx/2), sigma=3)]),
    'v': np.concatenate([homogeneous(0, nx)]),
    'backend': {}
    }

experiment.add_solver('ode', **parameters_1)
experiment.add_solver('local', **parameters_1)
results = experiment.run()

2023-11-23 12:16:36,548 - INFO - Created experiment with time stamp: 20231123T121636.

2023-11-23 12:16:36,549 - INFO - Adding solver 1: ode
2023-11-23 12:16:36,550 - INFO - Parameters checked for validity.
2023-11-23 12:16:36,550 - INFO - Solving for 15 time steps.
2023-11-23 12:16:36,550 - INFO - Calculating Transformation and Hamiltonian.
2023-11-23 12:16:36,557 - INFO - Calculation completed.
2023-11-23 12:16:36,619 - INFO - Medium initialized.
2023-11-23 12:16:36,626 - INFO - Initial state forward-transformed.
2023-11-23 12:16:36,627 - INFO - Solver 1 added.

2023-11-23 12:16:36,628 - INFO - Adding solver 2: local
2023-11-23 12:16:36,629 - INFO - Parameters checked for validity.
2023-11-23 12:16:36,630 - INFO - Solving for 15 time steps.
2023-11-23 12:16:36,632 - INFO - Calculating Transformation and Hamiltonian.
2023-11-23 12:16:36,643 - INFO - Calculation completed.
2023-11-23 12:16:36,644 - INFO - Medium initialized.
2023-11-23 12:16:36,646 - INFO - Initial state transformed.
2

In [7]:
idx = 10
u1 = results[0]['field']['u'][idx]
u2 = results[1]['field']['u'][idx]
v1 = results[0]['field']['v'][idx]
v2 = results[1]['field']['v'][idx]
t = results[0]['times'][idx]
plot_uv(u1, v1, t, plot_v=True)

In [8]:
plot_uv(u2, v2, t, plot_v=True)

In [9]:
plot_uv(u1-u2, v1-v2, t, plot_v=True)